<a href="https://colab.research.google.com/github/dantae74/machine-learning/blob/main/stock-etc/99-rnn-lotto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

모두를 위한 머신러닝에서 가져왔습니다.
# RNN-lotto

In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
url = 'https://raw.githubusercontent.com/dantae74/machine-learning/main/dataset/lotto.csv'
df = pd.read_csv(url)

In [3]:
df = df.sort_values('seq', ascending=True)
ncount = np.zeros((1,45), dtype=float)

In [4]:
for i in range(df.shape[0]):
  for j in range(6):
    col = df.iloc[i][str(j+1)]
    ncount[0][col-1] += 1.0

  if i == 0:
    X = ncount
  else:
    X = np.append(X, ncount, axis=0)

print('x.shape:', X.shape)

x.shape: (964, 45)


In [5]:
df_y = df[['1','2','3','4','5','6']]
Y = df_y.values
print('x.shape:', X.shape)
print('y.shape:', Y.shape)

x.shape: (964, 45)
y.shape: (964, 6)


In [6]:
y_onehot = []
tmponehot = np.zeros(45).tolist()
tmponehot[3] = 1.0
y_onehot.append(tmponehot)
y_onehot.append(tmponehot)

np.array(y_onehot).shape

(2, 45)

In [7]:
def make_onehot(value):
  onehot = np.zeros(45).tolist()
  onehot[value-1] = 1.0
  return onehot

In [8]:
y_onehot = []
for i in range(Y.shape[0]):
  for j in range(6):
    y_onehot.append(make_onehot(Y[i][j]))

y_onehot = np.array(y_onehot)
y_onehot = np.reshape(y_onehot, (964,6,45))
print('y_onehot.shape:', y_onehot.shape)

y_onehot.shape: (964, 6, 45)


In [9]:
X.shape

(964, 45)

In [10]:
def build_dataset(x_time_series, y_time_series, seq_length):
    dataX = []
    dataY = []
    for i in range(0, len(y_time_series) - seq_length):
        x = x_time_series[i:i + seq_length, :]
        y = y_time_series[i + seq_length - 1, :]
        dataX.append(x)
        dataY.append(y)
    return np.array(dataX), np.array(dataY)

In [11]:
x_data, y_data = build_dataset(X, y_onehot, 6)

In [12]:
print('x_data.shape:', x_data.shape)
print('y_data.shape:', y_data.shape)

x_data.shape: (958, 6, 45)
y_data.shape: (958, 6, 45)


In [13]:
model = keras.Sequential();
model.add(keras.layers.LSTM(units=45, input_shape=(6, 45), return_sequences=True))
model.add(keras.layers.LSTM(units=45, return_sequences=True))
model.add(keras.layers.Dense(units=45, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=0.01), metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 6, 45)             16380     
_________________________________________________________________
lstm_1 (LSTM)                (None, 6, 45)             16380     
_________________________________________________________________
dense (Dense)                (None, 6, 45)             2070      
Total params: 34,830
Trainable params: 34,830
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.fit(x_data, y_data, epochs=1200, verbose=0)

In [15]:
x_test = x_data[-1,:]
x_test = np.delete(x_test, 0, axis=0)
x_test = np.append(x_test, X[-1:], axis=0)
x_test = x_test.reshape((1,6,45))
print('x_test.shape:', x_test.shape)

x_test.shape: (1, 6, 45)


In [16]:
pred_y = model.predict(x_test)

In [ ]:
for i in range(6):
  print(np.argmax(pred_y[0][i]) + 1)